In [9]:
from backtesting import Backtest, Strategy
import pandas as pd
from stockstats import wrap 
from xbbg import blp

In [10]:

spx = blp.bdh(tickers=["SPX Index"], flds=["px_last", "px_high","px_low"], start_date = "2007-01-01")
spx.columns = spx.columns.droplevel()
spx.rename(columns={"px_last": "Close", "px_high": "High", "px_low": "Low"}, inplace=True)
# Dummy column for the wrapper
spx["Volume"] = 0
spx = wrap(spx)

# To get an RSI with a different period, change the parameter in spx.get('rsi') to spx.get('rsi_period')
# e.g. for a period of 2, do spx.get('rsi_2') - default is 14

rsi = spx.get('rsi')

vix = blp.bdh(tickers=["UX1 Index"], flds=["px_last", "px_high","px_low", "px_open"], start_date = "2007-01-01")
vix.columns = vix.columns.droplevel()
vix.rename(columns={"px_last": "Close", "px_high": "High", "px_low": "Low", "px_open": "Open"}, inplace=True)
df = vix.join(rsi, how='inner')
df.dropna(inplace=True)
df.drop(df[df["rsi"] == 100].index, inplace=True)


signal = pd.DataFrame(0, index=df.index, columns=["Signal"])
df = pd.concat([df, signal], axis=1)
rsi_90 = df.index[df["rsi"] > 80].to_list()
rsi_30 = df.index[df["rsi"] <= 59].to_list()
df.drop(df.columns[-2], axis=1, inplace=True)

df.loc[rsi_90, "Signal"] = 1
df.loc[rsi_30, "Signal"] = -1

df.index = pd.to_datetime(df.index)

df.head()

,Close,High,Low,Open,Signal
2007-01-05,12.38,12.44,12.19,12.19,-1
2007-01-08,12.25,12.56,12.21,12.42,-1
2007-01-09,12.34,12.50,12.10,12.28,-1
2007-01-10,11.90,12.56,11.90,12.48,-1
2007-01-11,10.98,11.78,10.80,11.78,0


In [11]:

class VIX_LS(Strategy):

    def init(self):
        pass
    def next(self):
        current_signal = self.data.Signal[-1]
        
        trades = self.trades
        def parse_trades(trades):
            for x in trades:
                a = x.entry_time
                b = self.data.index[-1]

                
                delta = b - a


                # Maximum hold period

                if delta.days > 1:
                    x.close()


        parse_trades(trades)
        if current_signal == 100:
        
            self.sell(size=1)
        
        elif current_signal == -1:
            
            self.buy(size=-100)

bt = Backtest(df, VIX_LS, cash=10000)
stats = bt.run()
print(stats)
bt.plot()

Start                     2007-01-05 00:00:00
End                       2024-07-22 00:00:00
Duration                   6408 days 00:00:00
Exposure Time [%]                   67.942909
Equity Final [$]                      42218.5
Equity Peak [$]                       42218.5
Return [%]                            322.185
Buy & Hold Return [%]               20.759289
Return (Ann.) [%]                    8.570197
Volatility (Ann.) [%]               39.542209
Sharpe Ratio                         0.216735
Sortino Ratio                        0.328576
Calmar Ratio                         0.125467
Max. Drawdown [%]                  -68.306157
Avg. Drawdown [%]                   -3.970462
Max. Drawdown Duration     2211 days 00:00:00
Avg. Drawdown Duration       48 days 00:00:00
# Trades                                 2288
Win Rate [%]                        59.134615
Best Trade [%]                       32.45283
Worst Trade [%]                    -67.153285
Avg. Trade [%]                    

GridPlot(id='p1841', ...)